In [1]:
import glob
from PIL import Image, ImageOps
from src.atom_seg import *
import cv2

# Label ids of the dataset
category_ids = {
    # "outlier": 0,
    "rebar": 1,
    "spall": 2,
    "crack": 3,
}

In [2]:
def add_border(input_image, output_image, border, color=0):
    img = Image.open(input_image)

    if isinstance(border, int) or isinstance(border, tuple):
        bimg = ImageOps.expand(img, border=border, fill=color)
    else:
        raise runtimeerror('border is not an integer or tuple!')

    return bimg #.save(output_image)

def add_margin(pil_img, top, right, bottom, left, color):
    width, height = pil_img.size
    new_width = width + right + left
    new_height = height + top + bottom
    result = Image.new(pil_img.mode, (new_width, new_height), color)
    result.paste(pil_img, (left, top))
    return result
    
def images_annotations_info(root_path):
    # This id will be automatically increased as we go
    image_id = 0
    annotations = []
    images = []

    for main_image in glob.glob(root_path + "images/"+ "*.jpg"):
        image_id+=1
        # print(main_image)
        
        main_img = Image.open(main_image)
        width, height = main_img.size
        # print("main_img.size", main_img.size)
        # The mask image is *.png but the original image is *.jpg.
        # We make a reference to the original file in the COCO JSON file
        filename = os.path.basename(main_image).split(".")[0]
        image = create_image_annotation(filename, width, height, image_id)
        images.append(image)
        rebarmask_file = os.path.join(root_path, "masks",filename + 'rebar' + '.jpg')
        spallmask_file = os.path.join(root_path, "masks", filename + 'spall' + '.jpg')
        crackmask_file = os.path.join(root_path, "masks", filename + 'crack' + '.jpg')
        
        # rebarmask = np.zeros_like(main_img)
        # spallmask = np.zeros_like(main_img)
        # crackmask = np.zeros_like(main_img)
        # Open the image and (to be sure) we convert it to RGB
        thresh = 150
        fn = lambda x : 255 if x > thresh else 0
        if os.path.exists(rebarmask_file):
            mask1 = cv2.imread(rebarmask_file)
            Conv_hsv_Gray = cv2.cvtColor(mask1, cv2.COLOR_BGR2GRAY)
            ret,rebarmask = cv2.threshold(Conv_hsv_Gray,127,255,cv2.THRESH_BINARY)
            category_id = category_ids["rebar"]
            annotations.append(create_annotation_format(rebarmask, category_id, image_id))
        if os.path.exists(spallmask_file):
            mask2 = cv2.imread(spallmask_file)#.convert("1")
            Conv_hsv_Gray = cv2.cvtColor(mask2, cv2.COLOR_BGR2GRAY)
            ret,spallmask = cv2.threshold(Conv_hsv_Gray,127,255,cv2.THRESH_BINARY)
            category_id = category_ids["spall"]
            annotations.append(create_annotation_format(spallmask, category_id, image_id))
        if os.path.exists(crackmask_file):
            mask3 = cv2.imread(crackmask_file)#.convert("1")
            Conv_hsv_Gray = cv2.cvtColor(mask3, cv2.COLOR_BGR2GRAY)
            ret,crackmask = cv2.threshold(Conv_hsv_Gray,127,255,cv2.THRESH_BINARY)
            category_id = category_ids["crack"]
            annotations.append(create_annotation_format(crackmask, category_id, image_id))
    # print(json.dumps(annotations, indent=4))
    return images, annotations, annotation_id
    

In [3]:
if __name__ == "__main__":
    # Get the standard COCO JSON format
    coco_format = get_coco_json_format()
    root = "../../data/"
    # root = "../../../Dataset/buildings/Building dataset/"
    
    for keyword in ["train"]:
        root_path = root+"dataset/"
        mask_path = root_path + "masks"
        
        # Create category section
        coco_format["categories"] = create_category_annotation(category_ids)
    
        # Create images and annotations sections
        coco_format["images"], coco_format["annotations"], annotation_cnt = images_annotations_info(root_path)
        
        with open("output/coco.json".format(keyword),"w") as outfile:
            json.dump(coco_format, outfile)
        
        print("Created %d annotations for images in folder: %s" % (annotation_cnt, mask_path))

Created 0 annotations for images in folder: ../../data/dataset/masks
